# 06 – Model Evaluation (PySpark)

This notebook evaluates the trained models on the test data using several metrics.  We compute area under the ROC curve (AUC) and accuracy, and derive a confusion matrix.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassificationModel, LogisticRegressionModel, GBTClassificationModel
from pyspark.mllib.evaluation import MulticlassMetrics
import os

spark = SparkSession.builder.appName('CTR_Model_Evaluation').getOrCreate()
processed_dir = os.path.join('..', 'data', 'processed')
models_dir = os.path.join('..', 'models')

# Load test data
test_df = spark.read.parquet(os.path.join(processed_dir, 'test_df.parquet'))

# Define evaluator
evaluator = BinaryClassificationEvaluator(labelCol='clk', rawPredictionCol='rawPrediction', metricName='areaUnderROC')

models_to_evaluate = {
    'LogisticRegression': LogisticRegressionModel.load(os.path.join(models_dir, 'LogisticRegression_spark_model')),
    'RandomForest': RandomForestClassificationModel.load(os.path.join(models_dir, 'RandomForest_spark_model')),
    'GBT': GBTClassificationModel.load(os.path.join(models_dir, 'GBT_spark_model'))
}

results = []

for name, model in models_to_evaluate.items():
    predictions = model.transform(test_df)
    auc = evaluator.evaluate(predictions)
    # Compute accuracy
    accuracy = predictions.filter(predictions['prediction'] == predictions['clk']).count() / float(predictions.count())
    results.append((name, auc, accuracy))
    print(f"Model: {name} – AUC: {auc:.4f}, Accuracy: {accuracy:.4f}")

# Confusion matrix for the best model
best_model_name, best_auc, _ = max(results, key=lambda x: x[1])
best_model = models_to_evaluate[best_model_name]
best_predictions = best_model.transform(test_df)
# Convert predictions to RDD of (prediction, label)
pred_and_labels = best_predictions.select('prediction', 'clk').rdd.map(lambda row: (row.prediction, row.clk))
metrics = MulticlassMetrics(pred_and_labels)
cm = metrics.confusionMatrix().toArray()

print(f"
Confusion matrix for {best_model_name}:
", cm)
